In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np

In [3]:
cd ..

C:\Projects\python\recommender


In [5]:
from datasets.torch_movielen import MovelenDataset, TorchMovielen10k

In [6]:
movielen = TorchMovielen10k('./inputs/ml-100k/u.data', user_min=4, item_min=4)

2019-07-15 16:27:58,417 - C:\Projects\python\recommender\utils.py - INFO - Read dataset in ./inputs/ml-100k/u.data
I0715 16:27:58.417632  7184 torch_movielen.py:45] Read dataset in ./inputs/ml-100k/u.data
2019-07-15 16:27:58,427 - C:\Projects\python\recommender\utils.py - INFO - Original user size: 943
I0715 16:27:58.427594  7184 torch_movielen.py:49] Original user size: 943
2019-07-15 16:27:58,431 - C:\Projects\python\recommender\utils.py - INFO - Original item size: 1682
I0715 16:27:58.431582  7184 torch_movielen.py:50] Original item size: 1682
2019-07-15 16:27:58,436 - C:\Projects\python\recommender\utils.py - INFO - Filter user size: 943
I0715 16:27:58.436569  7184 torch_movielen.py:56] Filter user size: 943
2019-07-15 16:27:58,438 - C:\Projects\python\recommender\utils.py - INFO - Filter item size: 1413
I0715 16:27:58.438564  7184 torch_movielen.py:57] Filter item size: 1413
c:\projects\python\recommender\.venv\lib\site-packages\pandas\core\generic.py:8682: SettingWithCopyWarning:

In [33]:
train_dl = movielen.get_dataloader(dataset_type='train', batch_size=16, num_workers=0)
valid_dl = movielen.get_dataloader(dataset_type='valid', batch_size=16, num_workers=0)
test_dl = movielen.get_dataloader(dataset_type='test', batch_size=16, num_workers=0)
train_it = iter(train_dl)

In [8]:
pos_batch, neg_batch = train_it.next()

In [32]:
import torch as T
import torch.nn as nn
import torch.optim as optim

In [16]:
cat_dict = movielen.cat_dict
pos_names = movielen.pos_cat_names
neg_names = movielen.neg_cat_names
num_dim = 32

In [19]:
from models.torch_fm import TorchFM, FMLearner

In [20]:
model = TorchFM(cat_dict, pos_names, neg_names, num_dim=10)
model

TorchFM(
  (emb_linear_layer): ModuleList(
    (0): Embedding(943, 1)
    (1): Embedding(1413, 1)
    (2): Embedding(1415, 1)
  )
  (emb_factor_layer): ModuleList(
    (0): Embedding(943, 10)
    (1): Embedding(1413, 10)
    (2): Embedding(1415, 10)
  )
)

In [17]:
emb_linear_layer = nn.ModuleList(
    [nn.Embedding(cat_dict[name].size, 1) for name in pos_names])
emb_factor_layer = nn.ModuleList(
    [nn.Embedding(cat_dict[name].size, num_dim) for name in pos_names])

In [26]:
linear = model.compute_linear_term(pos_batch)
linear

tensor([[ 0.5324],
        [ 1.8718],
        [-2.5095],
        [ 0.8001],
        [ 1.7884],
        [-1.0586],
        [-1.9556],
        [-0.9571],
        [ 0.0797],
        [-0.9875],
        [-0.1532],
        [-1.1193],
        [ 0.4469],
        [-0.9526],
        [ 0.9001],
        [ 0.1133]], grad_fn=<SumBackward2>)

In [25]:
factor = model.compute_factor_term(pos_batch)
factor

tensor([[ -6.9910],
        [-25.7246],
        [ 24.0390],
        [  3.9410],
        [  2.9823],
        [ -9.8013],
        [-13.7755],
        [ -2.4615],
        [ 11.5484],
        [ 22.1020],
        [ 10.7309],
        [ 11.4938],
        [ 10.1128],
        [ -0.9279],
        [ -2.8113],
        [ -2.5338]], grad_fn=<MulBackward0>)

In [27]:
preds = linear + factor
preds

tensor([[ -6.4586],
        [-23.8528],
        [ 21.5295],
        [  4.7412],
        [  4.7707],
        [-10.8599],
        [-15.7311],
        [ -3.4186],
        [ 11.6281],
        [ 21.1145],
        [ 10.5777],
        [ 10.3746],
        [ 10.5597],
        [ -1.8805],
        [ -1.9112],
        [ -2.4205]], grad_fn=<AddBackward0>)

In [28]:
preds.squeeze()

tensor([ -6.4586, -23.8528,  21.5295,   4.7412,   4.7707, -10.8599, -15.7311,
         -3.4186,  11.6281,  21.1145,  10.5777,  10.3746,  10.5597,  -1.8805,
         -1.9112,  -2.4205], grad_fn=<SqueezeBackward0>)

In [29]:
preds.size(0)

16

In [30]:
pos_preds, neg_preds = model(pos_batch, neg_batch)

In [37]:
learner = FMLearner(model, optim.Adam(model.parameters()), train_dl, valid_dl, test_dl)

In [38]:
bprloss = learner.criterion(pos_preds, neg_preds)
bprloss

c:\projects\python\recommender\.venv\lib\site-packages\torch\nn\functional.py:1386: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


tensor([30.7725], grad_fn=<NegBackward>)

In [39]:
learner.metric(pos_batch, pos_preds, neg_preds)

tensor(0)

In [52]:
binary_ranks = (pos_preds - neg_preds) > 0
binary_ranks = binary_ranks.to(T.float)
binary_ranks

tensor([0., 0., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 1.])

In [53]:
users_index = pos_batch[:, 0]

In [54]:
users, user_counts = T.unique(users_index, return_counts=True)

In [55]:
print("users: {}".format(users))
print("user counts: {}".format(user_counts))

users: tensor([ 32,  48,  56, 233, 307, 392, 425, 436, 510, 534, 628, 675, 701, 822,
        891, 932])
user counts: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])


In [56]:
users_index

tensor([675, 932,  48, 307, 628, 436, 701, 510, 233, 891, 392, 425,  56, 534,
         32, 822])

In [57]:
from torch_scatter import scatter_mean

In [58]:
auc_per_user = scatter_mean(binary_ranks, users_index)
auc_per_user

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 

In [59]:
T.sum(auc_per_user) / users.size(0)

tensor(0.6875)